In [1]:
import numpy as np
import serial
import time
from time import sleep

In [2]:
ser = serial.Serial(port='COM4',baudrate=115200,timeout=.2)
print(ser.name)

COM4


In [3]:
def serialQry(port, command):
    port.reset_input_buffer()
    port.write((command + '\r').encode('ascii'))
    return port.readline().decode('ascii').strip() #strip removes white space

def serialCmd(port, command):
    port.reset_input_buffer()
    port.write((command + '\r').encode('ascii'))

In [4]:
serialQry(ser, '*IDN?')

'Stanford_Research_Systems, SR542, s/n00000001, v0.10'

In [31]:
serialCmd(ser, '*CLS')

## Set up the instrument

In [5]:
serialCmd(ser, 'TOKN ON')
serialQry(ser, 'TOKN?')

'ON'

In [6]:
# monitor CHCR for speed lock acquisition
serialCmd(ser, 'CHPT 4')
serialQry(ser, 'CHPT?')

'4'

In [7]:
serialCmd(ser, 'SRCE INT')
serialQry(ser, 'SRCE?')

'INT'

In [8]:
serialCmd(ser, 'CTRL SHAFT')
serialQry(ser, 'CTRL?')

'SHAFT'

In [9]:
f_start = 80
serialCmd(ser, f'IFRQ {f_start}')
serialQry(ser, 'IFRQ?')

'80.00'

In [10]:
serialQry(ser, 'CHEV?')

'0'

In [11]:
# Make sure chopper is in CAL mode
serialCmd(ser, 'BSRS ON, 314159265')
serialQry(ser, 'BSRS?')

'ON'

In [12]:
# OPTIONAL: Set to run in UNCAL mode
serialCmd(ser, '$UCL OFF')
serialQry(ser, '$UCL?')

'OFF'

In [13]:
# Check CAL status
serialCmd(ser, '$CHT DNLD')

In [14]:
serialQry(ser, '$CSR?')

'20460001'

In [15]:
serialQry(ser, '$CIR?')

'65'

In [16]:
serialQry(ser, '$TIK? 399')

'9.975164e-01'

In [17]:
serialQry(ser, '$CCR? 399')

'1.776127e-03'

In [18]:
serialQry(ser, 'LERR?')

'0'

## Run the motor up to speed

In [75]:
serialQry(ser, 'KPHA?')

'3.0000, 3.0000, 1.0000'

In [18]:
serialQry(ser, '$UCL?')

'OFF'

In [51]:
serialCmd(ser, '$UCL OFF')

In [51]:
serialQry(ser, 'CHEV?')

'0'

In [20]:
serialCmd(ser, 'MOTR ON')

while True:
    if(int(serialQry(ser, 'CHEV?')) & 4):
        # perform the spin-down measurement after the motor achieves speed lock
        sleep(10.0)
        serialCmd(ser, '$SDN OUTER') # choose between OUTER, INNER, and SHAFT to input capture edges from that track
        break
    else:
        sleep(1.0)

## Record the spin-down measurement data to a text file

In [21]:
from pathlib import Path
import os
from glob import glob

In [22]:
file_dir = Path('.\CalData')

if not os.path.exists(file_dir):
    os.makedirs(file_dir)

In [23]:
serialQry(ser, 'SLOT?')

'5, 6'

In [55]:
serialQry(ser, '$SDA? 3')

'0, 1402356981, 0.000000e+00'

In [24]:
def spinDown_ArrayRecord(saveToDirectory):
    numPts = 14648 # for high speed, high inertia encoder cal
    #numPts = 1024 # for low speed, low inertia torque cal
    encCount = np.zeros(numPts)
    ftmCount = np.zeros(numPts)
    phase = np.zeros(numPts)
    
    i = 0
    for i in range(numPts):
        result = serialQry(ser, '$SDA? '+str(i)).split(', ')
        encCount[i] = np.uint32(result[0])
        ftmCount[i] = np.uint32(result[1])
        phase[i] = np.float32(result[2]) 
        
    sn = serialQry(ser, '$CSR?')
    shaftIndex = serialQry(ser, '$CIR?')
    rotorOffset = serialQry(ser, '$ROF?')
    
    hdr = f'sn = {sn}\nshaft index = {shaftIndex}\nrotor offset = {rotorOffset}\n'
    
    file_dir = Path(saveToDirectory)
    filesForThisSn = file_dir.glob(f'torqueCal_{sn}_*.txt')
    trialNum = sum(1 for file in filesForThisSn)
    
    fname = f'bladeCal_{sn}_outer_{trialNum}.txt'
    
    np.savetxt((file_dir / fname),
               np.transpose([encCount, ftmCount, phase]), header = hdr, newline='\n', delimiter=',')

In [25]:
# TODO: get chopper serial number from chopper head ($CHT)
# TODO: write file name programatically
t0 = time.time()
spinDown_ArrayRecord('.\CalData\Accel Measurements')
tf = time.time()
dt = tf - t0
print(f'elapsed time = {dt}')

elapsed time = 234.25864624977112


In [108]:
serialQry(ser, 'MWAN?')

'0'

## Process this data with encoderInputCapture.py

In [54]:
serialCmd(ser, '$IMQ 0.7')

In [84]:
serialCmd(ser, '$SHC')

In [646]:
serialQry(ser, 'LERR?')

'104'

In [89]:
serialCmd(ser, 'MOTR 0')